<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary 代码 for 这个 <一个 href="http://mng.bz/orYv">构建 一个 大语言模型 From Scratch</一个> book by <一个 href="https://sebastianraschka.com">Sebastian Raschka</一个><br>
<br>代码 repository: <一个 href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</一个>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<一个 href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></一个>
</td>
</tr>
</table>

# Extending 这个 Tiktoken BPE 分词器 with New Tokens

- 这个 笔记本 explains 如何 我们 can extend 一个 existing BPE 分词器; specifically, 我们 will focus on 如何 to do 它 for 这个 popular [tiktoken](https://github.com/openai/tiktoken) 实现
- For 一个 general 介绍 to tokenization, please refer to [第 2](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-第-代码/ch02.ipynb) 和 这个 BPE from Scratch [link] 教程
- For 示例, suppose 我们 have 一个 GPT-2 分词器 和 want to encode 这个 following text:

In [1]:
import tiktoken

base_tokenizer = tiktoken.get_encoding("gpt2")
sample_text = "Hello, MyNewToken_1 is a new token. <|endoftext|>"

token_ids = base_tokenizer.encode(sample_text, allowed_special={"<|endoftext|>"})
print(token_ids)

[15496, 11, 2011, 3791, 30642, 62, 16, 318, 257, 649, 11241, 13, 220, 50256]


- Iterating over each 词元 ID can give us 一个 better understanding of 如何 这个 词元 IDs are decoded via 这个 vocabulary:

In [2]:
for token_id in token_ids:
    print(f"{token_id} -> {base_tokenizer.decode([token_id])}")

15496 -> Hello
11 -> ,
2011 ->  My
3791 -> New
30642 -> Token
62 -> _
16 -> 1
318 ->  is
257 ->  a
649 ->  new
11241 ->  token
13 -> .
220 ->  
50256 -> <|endoftext|>


- As 我们 can see above, 这个 `"MyNewToken_1"` is broken down into 5 individual subword tokens -- 这个 is normal behavior for BPE 当 handling unknown words
- However, suppose 那个 它's 一个 special 词元 那个 我们 want to encode as 一个 single 词元, similar to some of 这个 other words 或者 `"<|endoftext|>"`; 这个 笔记本 explains 如何

&nbsp;
## 1. Adding special tokens

- Note 那个 我们 have to 添加 new tokens as special tokens; 这个 reason is 那个 我们 don't have 这个 "merges" for 这个 new tokens 那个 are created during 这个 分词器 训练 处理 -- even 如果 我们 had them, 它 would be very challenging to incorporate them without breaking 这个 existing tokenization scheme (see 这个 BPE from scratch 笔记本 [link] to understand 这个 "merges")
- Suppose 我们 want to 添加 2 new tokens:

In [3]:
# 定义 custom tokens 和 their 词元 IDs
custom_tokens = ["MyNewToken_1", "MyNewToken_2"]
custom_token_ids = {
    token: base_tokenizer.n_vocab + i for i, token in enumerate(custom_tokens)
}

- 接下来, 我们 创建 一个 custom `Encoding` object 那个 holds our special tokens as follows:

In [4]:
# 创建 一个 new Encoding object with extended tokens
extended_tokenizer = tiktoken.Encoding(
    name="gpt2_custom",
    pat_str=base_tokenizer._pat_str,
    mergeable_ranks=base_tokenizer._mergeable_ranks,
    special_tokens={**base_tokenizer._special_tokens, **custom_token_ids},
)

- 那个's 它, 我们 can 现在 检查 那个 它 can encode 这个 sample text:

- As 我们 can see, 这个 new tokens `50257` 和 `50258` are 现在 encoded in 这个 输出:

In [5]:
special_tokens_set = set(custom_tokens) | {"<|endoftext|>"}

token_ids = extended_tokenizer.encode(
    "Sample text with MyNewToken_1 and MyNewToken_2. <|endoftext|>",
    allowed_special=special_tokens_set
)
print(token_ids)

[36674, 2420, 351, 220, 50257, 290, 220, 50258, 13, 220, 50256]


- Again, 我们 can also look at 它 on 一个 per-词元 level:

In [6]:
for token_id in token_ids:
    print(f"{token_id} -> {extended_tokenizer.decode([token_id])}")

36674 -> Sample
2420 ->  text
351 ->  with
220 ->  
50257 -> MyNewToken_1
290 ->  and
220 ->  
50258 -> MyNewToken_2
13 -> .
220 ->  
50256 -> <|endoftext|>


- As 我们 can see above, 我们 have successfully updated 这个 分词器
- However, to 使用 它 with 一个 pretrained 大语言模型, 我们 also have to 更新 这个 嵌入 和 输出 layers of 这个 大语言模型, 哪个 is discussed in 这个 接下来 section

&nbsp;
## 2. Updating 一个 pretrained 大语言模型

- In 这个 section, 我们 will take 一个 look at 如何 我们 have to 更新 一个 existing pretrained 大语言模型 after updating 这个 分词器
- For 这个, 我们 are using 这个 original pretrained GPT-2 模型 那个 is used in 这个 main book

&nbsp;
### 2.1 Loading 一个 pretrained GPT 模型

In [7]:
from llms_from_scratch.ch05 import download_and_load_gpt2
# For llms_from_scratch 安装 instructions, see:
# https://github.com/rasbt/LLMs-from-scratch/tree/main/pkg

settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")

checkpoint: 100%|███████████████████████████| 77.0/77.0 [00:00<00:00, 34.4kiB/s]
encoder.json: 100%|███████████████████████| 1.04M/1.04M [00:00<00:00, 4.78MiB/s]
hparams.json: 100%|█████████████████████████| 90.0/90.0 [00:00<00:00, 24.7kiB/s]
model.ckpt.data-00000-of-00001: 100%|███████| 498M/498M [00:33<00:00, 14.7MiB/s]
model.ckpt.index: 100%|███████████████████| 5.21k/5.21k [00:00<00:00, 1.05MiB/s]
model.ckpt.meta: 100%|██████████████████████| 471k/471k [00:00<00:00, 2.33MiB/s]
vocab.bpe: 100%|████████████████████████████| 456k/456k [00:00<00:00, 2.45MiB/s]


In [8]:
from llms_from_scratch.ch04 import GPTModel
# For llms_from_scratch 安装 instructions, see:
# https://github.com/rasbt/LLMs-from-scratch/tree/main/pkg

GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # 嵌入 dimension
    "n_heads": 12,         # Number of 注意力机制 heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value 偏置
}

# 定义 模型 configurations in 一个 dictionary for compactness
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

# Copy 这个 base 配置 和 更新 with specific 模型 settings
model_name = "gpt2-small (124M)"  # 示例 模型 name
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})

gpt = GPTModel(NEW_CONFIG)
gpt.eval();

### 2.2 Using 这个 pretrained GPT 模型

- 接下来, consider our sample text below, 哪个 我们 tokenize using 这个 original 和 这个 new 分词器:

In [9]:
sample_text = "Sample text with MyNewToken_1 and MyNewToken_2. <|endoftext|>"

original_token_ids = base_tokenizer.encode(
    sample_text, allowed_special={"<|endoftext|>"}
)

In [10]:
new_token_ids = extended_tokenizer.encode(
    "Sample text with MyNewToken_1 and MyNewToken_2. <|endoftext|>",
    allowed_special=special_tokens_set
)

- 现在, 让我们 feed 这个 original 词元 IDs to 这个 GPT 模型:

In [11]:
import torch

with torch.no_grad():
    out = gpt(torch.tensor([original_token_ids]))

print(out)

tensor([[[ 0.2204,  0.8901,  1.0138,  ...,  0.2585, -0.9192, -0.2298],
         [ 0.6745, -0.0726,  0.8218,  ..., -0.1768, -0.4217,  0.0703],
         [-0.2009,  0.0814,  0.2417,  ...,  0.3166,  0.3629,  1.3400],
         ...,
         [ 0.1137, -0.1258,  2.0193,  ..., -0.0314, -0.4288, -0.1487],
         [-1.1983, -0.2050, -0.1337,  ..., -0.0849, -0.4863, -0.1076],
         [-1.0675, -0.5905,  0.2873,  ..., -0.0979, -0.8713,  0.8415]]])


- As 我们 can see above, 这个 works without problems (note 那个 这个 代码 shows 这个 raw 输出 without converting 这个 outputs back into text for simplicity; for more details on 那个, please 检查 out 这个 `生成` 函数 in 第 5 [link] section 5.3.3

- 什么 happens 如果 我们 try 这个 same on 这个 词元 IDs generated by 这个 updated 分词器 现在?

```python
with torch.no_grad():
    GPT(torch.tensor([new_token_ids]))

打印(out)

...
# IndexError: index out of range in self
```

- As 我们 can see, 这个 results in 一个 index error
- 这个 reason is 那个 这个 GPT 模型 expects 一个 fixed vocabulary size via its 输入 嵌入 层 和 its 输出 层:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/extend-tiktoken/GPT-updates.webp" width="400px">

&nbsp;
### 2.3 Updating 这个 嵌入 层

- 让我们 开始 with updating 这个 嵌入 层
- 首先, notice 那个 这个 嵌入 层 has 50,257 entries, 哪个 corresponds to 这个 vocabulary size:

In [12]:
gpt.tok_emb

Embedding(50257, 768)

- 我们 want to extend 这个 嵌入 层 by adding 2 more entries
- In short, 我们 创建 一个 new 嵌入 层 with 一个 bigger size, 和 然后 我们 copy over 这个 old 嵌入 层 values

In [13]:
num_tokens, emb_size = gpt.tok_emb.weight.shape
new_num_tokens = num_tokens + 2

# 创建 一个 new 嵌入 层
new_embedding = torch.nn.Embedding(new_num_tokens, emb_size)

# Copy weights from 这个 old 嵌入 层
new_embedding.weight.data[:num_tokens] = gpt.tok_emb.weight.data

# Replace 这个 old 嵌入 层 with 这个 new one in 这个 模型
gpt.tok_emb = new_embedding

print(gpt.tok_emb)

Embedding(50259, 768)


- As 我们 can see above, 我们 现在 have 一个 increased 嵌入 层

&nbsp;
### 2.4 Updating 这个 输出 层

- 接下来, 我们 have to extend 这个 输出 层, 哪个 has 50,257 输出 features corresponding to 这个 vocabulary size similar to 这个 嵌入 层 (by 这个 way, 你 may find 这个 bonus material, 哪个 discusses 这个 similarity between Linear 和 嵌入 layers in PyTorch, useful)

In [14]:
gpt.out_head

Linear(in_features=768, out_features=50257, bias=False)

- 这个 procedure for extending 这个 输出 层 is similar to extending 这个 嵌入 层:

In [15]:
original_out_features, original_in_features = gpt.out_head.weight.shape

# 定义 这个 new number of 输出 features (e.g., adding 2 new tokens)
new_out_features = original_out_features + 2

# 创建 一个 new linear 层 with 这个 extended 输出 size
new_linear = torch.nn.Linear(original_in_features, new_out_features)

# Copy 这个 weights 和 biases from 这个 original linear 层
with torch.no_grad():
    new_linear.weight[:original_out_features] = gpt.out_head.weight
    if gpt.out_head.bias is not None:
        new_linear.bias[:original_out_features] = gpt.out_head.bias

# Replace 这个 original linear 层 with 这个 new one
gpt.out_head = new_linear

print(gpt.out_head)

Linear(in_features=768, out_features=50259, bias=True)


- 让我们 try 这个 updated 模型 on 这个 original 词元 IDs 首先:

In [16]:
with torch.no_grad():
    output = gpt(torch.tensor([original_token_ids]))
print(output)

tensor([[[ 0.2267,  0.9132,  1.0494,  ..., -0.2330, -0.3008, -1.1458],
         [ 0.6808, -0.0495,  0.8574,  ...,  0.0671,  0.5572, -0.7873],
         [-0.1947,  0.1045,  0.2773,  ...,  1.3368,  0.8479, -0.9660],
         ...,
         [ 0.1200, -0.1027,  2.0549,  ..., -0.1519, -0.2096,  0.5651],
         [-1.1920, -0.1819, -0.0981,  ..., -0.1108,  0.8435, -0.3771],
         [-1.0612, -0.5674,  0.3229,  ...,  0.8383, -0.7121, -0.4850]]])


- 接下来, 让我们 try 它 on 这个 updated tokens:

In [17]:
with torch.no_grad():
    output = gpt(torch.tensor([new_token_ids]))
print(output)

tensor([[[ 0.2267,  0.9132,  1.0494,  ..., -0.2330, -0.3008, -1.1458],
         [ 0.6808, -0.0495,  0.8574,  ...,  0.0671,  0.5572, -0.7873],
         [-0.1947,  0.1045,  0.2773,  ...,  1.3368,  0.8479, -0.9660],
         ...,
         [-0.0656, -1.2451,  0.7957,  ..., -1.2124,  0.1044,  0.5088],
         [-1.1561, -0.7380, -0.0645,  ..., -0.4373,  1.1401, -0.3903],
         [-0.8961, -0.6437, -0.1667,  ...,  0.5663, -0.5862, -0.4020]]])


- As 我们 can see, 这个 模型 works on 这个 extended 词元 设置
- In practice, 我们 want to 现在 finetune (或者 continually pretrain) 这个 模型 (specifically 这个 new 嵌入 和 输出 layers) on data containing 这个 new tokens

**一个 note about 权重 tying**

- 如果 这个 模型 uses 权重 tying, 哪个 means 那个 这个 嵌入 层 和 输出 层 share 这个 same weights, similar to Llama 3 [link], updating 这个 输出 层 is much simpler
- In 这个 case, 我们 can simply copy over 这个 weights from 这个 嵌入 层:

In [18]:
gpt.out_head.weight = gpt.tok_emb.weight

In [19]:
with torch.no_grad():
    output = gpt(torch.tensor([new_token_ids]))